In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/home/shawaiz/Desktop/Data_sets/covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [5]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
58,23,Male,98.0,Strong,Mumbai
75,5,Male,102.0,Mild,Kolkata
60,24,Female,102.0,Strong,Bangalore
16,69,Female,103.0,Mild,Kolkata
9,64,Female,101.0,Mild,Delhi
...,...,...,...,...,...
71,75,Female,104.0,Strong,Delhi
54,60,Female,99.0,Mild,Mumbai
34,74,Male,102.0,Mild,Mumbai
37,55,Male,100.0,Mild,Kolkata


# Without Column-Transformer

In [8]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape

(80, 1)

In [9]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [10]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [11]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [12]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

#X_train_transformed
X_train_transformed.shape
#pd.DataFrame(X_train_transformed)

(80, 7)

# With Column-Transforamtion

In [13]:
from sklearn.compose import ColumnTransformer

In [14]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),                                 # transformer_1 #
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),  # transformer_2 #
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city']) # transformer_3 #
],remainder='passthrough')

In [15]:
#transformer.fit_transform(X_train)
print(transformer.fit_transform(X_train).shape)

(80, 7)


In [16]:
transformer.transform(X_test).shape

(20, 7)